In [1]:
input1 = "/data/students/bigdata-01QYD/Lab7/register.csv"
input2 = "/data/students/bigdata-01QYD/Lab7/stations.csv"
threshold  = 0.4
output = "Spark_Output/Output_Lab8"

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
from pyspark.sql.types import DoubleType, IntegerType

In [4]:
df_register = spark.read.load(input1,
format="csv",
header=True,
inferSchema=True,
delimiter = "\\t")

In [5]:
df_register.show(5)

+-------+-------------------+----------+----------+
|station|          timestamp|used_slots|free_slots|
+-------+-------------------+----------+----------+
|      1|2008-05-15 12:01:00|         0|        18|
|      1|2008-05-15 12:02:00|         0|        18|
|      1|2008-05-15 12:04:00|         0|        18|
|      1|2008-05-15 12:06:00|         0|        18|
|      1|2008-05-15 12:08:00|         0|        18|
+-------+-------------------+----------+----------+
only showing top 5 rows



In [6]:
df_filtered = df_register.filter("free_slots != 0 or used_slots != 0")

In [7]:
df_filtered.show(5)

+-------+-------------------+----------+----------+
|station|          timestamp|used_slots|free_slots|
+-------+-------------------+----------+----------+
|      1|2008-05-15 12:01:00|         0|        18|
|      1|2008-05-15 12:02:00|         0|        18|
|      1|2008-05-15 12:04:00|         0|        18|
|      1|2008-05-15 12:06:00|         0|        18|
|      1|2008-05-15 12:08:00|         0|        18|
+-------+-------------------+----------+----------+
only showing top 5 rows



In [8]:
def checkCriticality(x):
    if x == 0:
        return 1.
    return 0.

In [9]:
#user defined function
spark.udf.register("check_criticality", checkCriticality, DoubleType())

<function __main__.checkCriticality(x)>

In [10]:
#Use of the defined UDF in a selectExpr transformation
resDF= df_filtered.selectExpr("station", "date_format(timestamp,'EE') as weekday",\
 "hour(timestamp) as hour", "check_criticality(free_slots) as critical")

In [11]:
#Use of the defined UDF in a SQL query
#I have to create a table before using spark SQL
df_filtered.createOrReplaceTempView("table")
result = spark.sql("SELECT station, date_format(timestamp,'EE') as weekday, hour(timestamp) as hour, check_criticality(free_slots) as critical FROM table")

In [12]:
resDF.show(5)

+-------+-------+----+--------+
|station|weekday|hour|critical|
+-------+-------+----+--------+
|      1|    Thu|  12|     0.0|
|      1|    Thu|  12|     0.0|
|      1|    Thu|  12|     0.0|
|      1|    Thu|  12|     0.0|
|      1|    Thu|  12|     0.0|
+-------+-------+----+--------+
only showing top 5 rows



[Stage 4:>                                                          (0 + 1) / 1]24/12/14 10:57:01 WARN datasources.FileScanRDD: Skipped the rest of the content in the corrupted file: path: hdfs://BigDataHa/data/students/bigdata-01QYD/Lab7/register.csv, range: 0-91222189, partition values: [empty row]
java.io.IOException: Stream closed
	at org.apache.hadoop.hdfs.DFSInputStream.readWithStrategy(DFSInputStream.java:744)
	at org.apache.hadoop.hdfs.DFSInputStream.read(DFSInputStream.java:829)
	at java.io.DataInputStream.read(DataInputStream.java:149)
	at org.apache.hadoop.mapreduce.lib.input.UncompressedSplitLineReader.fillBuffer(UncompressedSplitLineReader.java:62)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:218)
	at org.apache.hadoop.util.LineReader.readLine(LineReader.java:176)
	at org.apache.hadoop.mapreduce.lib.input.UncompressedSplitLineReader.readLine(UncompressedSplitLineReader.java:94)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.nextKeyValue

In [13]:
#group by avg
grouped_df = resDF.groupBy("station", "weekday", "hour").avg("critical")

In [14]:
#grouped_df.show(5)

In [15]:
renamed_df = grouped_df.withColumnRenamed("avg(critical)", "average")

In [16]:
groupedAboveThreshold_df = renamed_df.filter(f"average > {threshold}")

In [17]:
#groupedAboveThreshold_df.show()

In [18]:
sorted_df = groupedAboveThreshold_df.sort("station", "weekday", "hour",  ascending=True).sort("average",  ascending=False)

In [19]:
#sorted_df.show()

In [20]:
df_stations = spark.read.option("delimiter", "\\t").load(input2,
format="csv",
header=True,
inferSchema=True)

In [21]:
#df_stations.show(5)

In [22]:
df_register_stations= sorted_df.join(df_stations, sorted_df.station == df_stations.id).selectExpr("station", "weekday", "hour", "average", "longitude", "latitude")

In [23]:
#df_register_stations.show()

In [26]:
#df_register_stations.write.csv(output, header=True)
df_register_stations.rdd.saveAsTextFile(outputPath)


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46655)
Traceback (most recent call last):
  File "/opt/cloudera/parcels/CDH/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/cloudera/parcels/CDH/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:46655)